In [ ]:
# Selenium 사용해서 크롤링
# !pip install selenium (버전 업그레이드 후 충돌 나면 uninstall 후 다시 install, 커널 초기화)

# openai chatGPT API 활용하여 기사 본문 요약 받아옴
# !pip install openai

# openpyxl 사용해서 엑셀 파일로 내용 저장
# !pip install openpyxl

# Google Drive API 활용하여 저장된 엑셀 파일 특정 폴더에 업로드
# Google Clinet Library Install
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


from __future__ import print_function
from datetime import datetime

import time
import os.path

import openpyxl
import openai

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

# By 사용법
# By.ID By.NAME By.XPATH
# By.LINK_TEXT(링크 텍스트값으로 추출) By.PARTIAL_LINK_TEXT(링크 텍스트의 자식 텍스트 값을 추출)
# By.TAG_NAME By.CLASS_NAME By.CSS_SELECTOR

# 창 켜지 않고 웹 드라이버 실행 옵션
options = webdriver.ChromeOptions()
options.add_argument("--headless")

# 웹드라이버 초기화 (새 창 열림)
driver = webdriver.Chrome(options=options)

# 크롬 드라이버 버전 안 맞을 시 맞는 버전으로 다운로드하여 같은 폴더 안에 위치시키기
# https://chromedriver.chromium.org/downloads

# 날짜 입력하기
now = datetime.now()
today_str = now.strftime('%Y%m%d')

# GPT 3.5 API 키 설정
openai.api_key = "..."

# 한국경제 신문보기 페이지로 이동
page_url = 'https://media.naver.com/press/015/newspaper'
# (네이버 뉴스에 있는 '신문보기' 서비스를 제공하는 언론사라면 다른 언론사로 해볼 수도 있을듯)
driver.get(page_url)

# 페이지 로드 대기
time.sleep(2)

# 날짜 바꾸기 (당일에 못한 경우 수정해서 써보자)
# calendar = driver.find_element(By.CLASS_NAME, "cal_prev_btn") # 전날 버튼
# calendar.click()
# print('첫번째 성공')
# calendar.click()
# print('두번째 성공')
# calendar.click()
# print('세번째 성공')
# calendar.click()
# print('네번째 성공')
# time.sleep(2)

# 모든 기사 요소 가져오기
article_elements = driver.find_elements(By.XPATH, '//ul[@class="newspaper_article_lst"]/li/a')

# 엑셀 워크북 생성하기
workbook = openpyxl.Workbook()
sheet = workbook.active
sheet.title = f'{today_str}'

# sheet에 header 추가하기
sheet.append(['기사 링크', '기사 제목', '기사 요약 (chatGPT)', '기사 본문'])

# 기사 반복하기
for article_element in article_elements :
    try :
        article_title = article_element.find_element(By.TAG_NAME, 'strong').text
        # print(article_title) # 디버깅용
    except:
        article_title = "article_title find_element 실패 (TAG_NAME is not 'strong')"
    try :
        article_url = article_element.get_attribute('href')
        # print(article_url) # 디버깅용
    except :
        article_url = "https://media.naver.com/press/015/newspaper"

    # article_element.click() # 클릭으로 하니까 자꾸 element가 새로 바뀌어서 기존 element가 유효하지 않음
    # 새 탭으로 기사 열기
    driver.execute_script('window.open('');')
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(article_url)
    time.sleep(2)

    try :
        article_content = driver.find_element(By.ID, 'newsct_article').text
    except :
        try :
            # 스포츠 기사 (HTML 구조가 다름)
            article_content = driver.find_element(By.ID, 'newsEndContents').text
        except Exception as e :
            article_content = '본문 찾지 못함'
            # article_summary = '요약 없음'
            print(f"Content Find Error occurred : {e}")
    
    # 제목에서 요약하기 걸러낼 키워드 추가
    excluded_words = ['[포토]', '[인사]', '[부고]', '[오늘의 arte]', '[알립니다]', '[모십니다]', '[모바일한경]', '[한경에세이]', '칼럼]']
    
    # 정치, 스포츠, 오피니언 카테고리 요약 제외를 위해 section 카테고리명 찾기
    try :
        article_section = driver.find_element(By.CLASS_NAME, 'media_end_categorize_item').text
    except :
        try :
            sports = driver.find_element(By.CLASS_NAME, 'logo_sports').get_attribute('href')
            if sports :
                article_section = '스포츠'
            else :
                article_section = "스포츠 아님"
        except Exception as e_s :
            article_section = '예외 발생'
            print(f"예외 발생 : 스포츠면 예상.. 예외 내역 -> {e_s}")
    
    prompt = f'아래 기사를 200토큰 이내의 한국어로 요약해줘:\n{article_content}'

    # 본문 없는 경우 굳이 요약할 필요가 없음 (돈이다..)
    if article_content == '본문 찾지 못함' :
        article_summary = '요약 없음'
    # 정치면과 스포츠면은 요약하지 않음
    elif article_section == '정치' or article_section == '스포츠' or article_section == '오피니언' :
        article_summary = '요약 없음'
    # 너무 짧아도 요약할 필요 없음
    elif len(article_content) < 700 :
        article_summary = '요약 없음'
    # excluded_words가 제목에 포함된 경우도 요약할 필요 없음
    elif any(word in article_title for word in excluded_words) :
        article_summary = '요약 없음'
    else :
        try :
            # GPT 3.5로 요약 생성하기
            response = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages=[
                    {"role": "system", "content": "너는 뉴스와 아티클 본문의 핵심을 요약하는 전문가야"},
                    {"role": "user", "content": prompt}
                ],
                # max_tokens = 200
            )

            # message에 잘 들어왔나 디버깅용
            # print(f'strip 전 content : {response.choices[0].message["content"]}')
            
            # 요약된 텍스트 저장하기
            if response.choices and response.choices[0].message['content']:
                # 요약된 텍스트 저장하기
                article_summary = response.choices[0].message['content'].strip()
            else:
                article_summary = '요약 생성 실패'       
            print(f'요약 내용 : {article_summary}')
            
        except Exception as e1 :
            print(f'GPT-3.5-turbo 토큰 한도 초과, 에러 코드 : {e1}')
            try :
                # turbo 모델의 token이 모자라서 안되는 경우 -> 16k 모델로 바꿔줌
                response = openai.ChatCompletion.create(
                    model='gpt-3.5-turbo-16k',
                    messages=[
                        {"role": "system", "content": "너는 뉴스와 아티클 본문의 핵심을 요약하는 전문가야"},
                        {"role": "user", "content": prompt}
                    ],
                    max_tokens = 200
                )
                # 요약된 텍스트 저장하기
                if response.choices and response.choices[0].message['content']:
                    # 요약된 텍스트 저장하기
                    article_summary = response.choices[0].message['content'].strip()
                else:
                    article_summary = '요약 생성 실패 (16k)'       
                print(f'요약 내용(16k) : {article_summary}')
                
            except Exception as e2 :
                # 그래도 안되면 그냥 실패로 가자
                article_summary = "본문 요약 실패"
                print(f'Summary Error Occurred : {e2}')

    # 디버깅용 프린트
    # print('기사 제목 : ', article_title)
    # print('기사 링크 : ', article_url)
    # print('기사 본문 : ', article_content)

    # 데이터 엑셀 시트에 추가하기
    sheet.append([article_url, article_title, article_summary, article_content])

    # driver.back() # click() 일땐 이거 사용
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

    time.sleep(2)

# 엑셀 파일 저장
workbook.save(f'{today_str}_기사_데이터_요약.xlsx')
driver.quit()

# 구글드라이브 연결
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive']
folder_id = '...'

def drive_upload():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('drive', 'v3', credentials=creds)
    
        request_body = {
            'name': f'{today_str}_기사_데이터_요약.xlsx', # 파일명
            'parents': [folder_id] # 업로드할 폴더 ID
        }
        media = MediaFileUpload(f'{today_str}_기사_데이터_요약.xlsx', resumable=True) # resumable : 업로드 재개 가능한지 여부
        file = service.files().create(body=request_body,media_body=media,fields='id').execute()
        print("Upload has Completed, File ID:",file.get('id'))
    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f'An error occurred: {error}')


if __name__ == '__main__':
    drive_upload()